## Volume Hand Control

In [ ]:
# ! pip install pycaw

In [1]:
import cv2 as cv
import time
import numpy as np
import mediapipe as mp
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [ ]:
# Constants
wCam , hCam = 1280 , 720 # Cam  Width , Height

In [ ]:
class volumeControler():
    def __init__(self):    
        devices = AudioUtilities.GetSpeakers()
        interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
        self.volume = interface.QueryInterface(IAudioEndpointVolume)

    def getVolumeRange(self):
        volRange = self.volume.GetVolumeRange()
        print("Volume Range : ",volRange)
        return volRange[0],volRange[1]
        
    def setVolume(self,volume):
        self.volume.SetMasterVolumeLevel(volume, None)

In [ ]:
# Module
class handDetector():
    def __init__(self , mode = False , maxHands = 2 , detectionCon = 0.5 , trackCon = 0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        self.mpHands= mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode , self.maxHands ,1, self.detectionCon , self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils
        
    def findHands(self,img,draw = True):
        imgRGB =  cv.cvtColor(img , cv.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw : self.mpDraw.draw_landmarks(img,handLms , self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self , img , handNo = 0 , draw = True):
        lmList = []
        if self.results.multi_hand_landmarks:
            
            myHand = self.results.multi_hand_landmarks[handNo]
            
            for id , lm in enumerate(myHand.landmark):
                    h, w ,c = img.shape
                    cx , cy = int(lm.x*w ) , int(lm.y * h)
                    lmList.append([id, cx, cy])
                    
                    if draw : cv.circle(img,(cx , cy) , 25 , (255,0,255) , cv.FILLED)
                    
        return lmList
        

In [6]:

cap = cv.VideoCapture(0)
cap.set(3,wCam) # Cam Width
cap.set(4,hCam) # Cam height

pTime = 0

detector = handDetector(detectionCon=0.7)
volControler = volumeControler()
minVol , maxVol = volControler.getVolumeRange()

while True:
    success , img = cap.read()
    img = detector.findHands(img)
    vol = 0 
    volBar = 400
    lmList = detector.findPosition(img , draw= False)
    # print("Tracked Fingers : ",lmList[4] , lmList[8])
    if(len(lmList)!=0):
        x1 , y1 = lmList[4][1] , lmList[4][2]
        x2 , y2 = lmList[8][1] , lmList[8][2]
        cx , cy = (x1 + x2) // 2 , (y1 + y2)//2 # Centers
        cv.circle(img , (x1 , y1) , 15 , (255,0,255) , cv.FILLED)
        cv.circle(img , (x2 , y2) , 15 , (255,0,255) , cv.FILLED)
        cv.line(img , (x1 , y1) , (x2,  y2) , (255,0,255) , 3) # Lined Line
        cv.circle(img , (cx , cy) , 15 , (255,0,255) , cv.FILLED) # Centered Circle
        length = math.hypot(x2 - x1 , y2-y1 ) # Length of the line (Distance between two points)
        # print("Line Length : ",length)
        
        # Out Function
        if length < 50:
            cv.circle(img , (cx , cy) , 15 , (0,255,0) , cv.FILLED) # Centered Circle
        # Converting the range of Line length to be equal to volume range
        vol = np.interp(length , [22,260] , [minVol,maxVol])  # Change [22,260] as it is suitable with lengh
        print(length , vol)
        volControler.setVolume(vol)
        volBar = np.interp(length , [22,260] , [400,150])  # Change [22,260] as it is suitable with lengh     
           
    cv.rectangle(img , (50 , 150) , (84,400),(0,255,0) , 3)
    cv.rectangle(img , (50 , int(volBar)) , (84,400),(0,255,0) , cv.FILLED)
    
    
    cTime = time.time()
    fbs = 1/(cTime - pTime)
    pTime = cTime
    cv.putText(img , f"FBS {int(fbs)}" ,(40,50) , cv.FONT_HERSHEY_COMPLEX , 2 , (255,0,0) , 2)
    
    cv.imshow("Image",img)
    cv.waitKey(1)

Volume Range :  (-96.0, 0.0, 0.125)
48.010415536631214 -85.50840381715716
80.06247560499239 -72.57984177277618
113.29607230614837 -59.174693523570404
138.15932831336434 -49.14581715091186
228.65913495856665 -12.641693462090757
239.70815588961506 -8.184945523516618
244.77336456403913 -6.141836142236315
248.2438317461282 -4.741983833494501
253.3081127796739 -2.699248626686156
258.0697580112788 -0.7785850038539337
257.42183279589943 -1.0399329898893086
254.9078264785136 -2.0539859582466136
253.3554814879678 -2.6801419208197075
250.79274311670184 -3.7138515159522
249.04015740438328 -4.420776845290774
249.6096953245206 -4.191047264058909
250.96015620014265 -3.646323549522293
256.51705596314645 -1.4048849896552156
250.85653270345583 -3.688121262471597
254.9078264785136 -2.0539859582466136
253.65330670030698 -2.5600107427333114
248.28411145298847 -4.725736556777761
251.09759058979438 -3.5908878293266326
245.70917768776974 -5.764365302412202
245.40578640284747 -5.886741619019503
202.2498454881

KeyboardInterrupt: 